# co

> Covariance Matrix and Coherence Matrix Estimation

In [ ]:
#| default_exp co

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import numpy as np
import itertools
from decorrelation.shp import ks_test
import math

In [ ]:
#| export
import cupy as cp

In [ ]:
_co_mat_kernel = cp.ElementwiseKernel(
    'raw T rslc, raw bool is_shp, int32 nlines, int32 width, int32 nimages, int32 az_half_win, int32 r_half_win',
    'raw T cov, raw T coh',
    '''
    if (i >= nlines*width) return;
    int az_win = 2*az_half_win+1;
    int r_win = 2*r_half_win+1;
    int win = az_win*r_win;
    
    int ref_az = i/width;
    int ref_r = i -ref_az*width;

    int sec_az, sec_r;

    int m,j; // index of each coherence matrix
    int k,l; // index of search window
    T _cov; // covariance
    float _amp2_m; // sum of amplitude square for image i
    float _amp2_j; // sum of amplitude aquare for image j
    int rslc_inx_m, rslc_inx_j;

    for (m = 0; m < nimages; m++) {
        for (j = 0; j < nimages; j++) {
            _cov = T(0.0, 0.0);
            _amp2_m = 0.0;
            _amp2_j = 0.0;
            for (k = 0; k < az_win; k++) {
                for (l = 0; l < r_win; l++) {
                    sec_az = ref_az-az_half_win+k;
                    sec_r = ref_r-r_half_win+l;
                    if (is_shp[i*win+k*r_win+l] && sec_az >= 0 && sec_az < nlines && sec_r >= 0 && sec_r < width) {
                        rslc_inx_m = (sec_az*width+sec_r)*nimages+m;
                        rslc_inx_j = (sec_az*width+sec_r)*nimages+j;
                        _amp2_m += norm(rslc[rslc_inx_m]);
                        _amp2_j += norm(rslc[rslc_inx_j]);
                        _cov += rslc[rslc_inx_m]*conj(rslc[rslc_inx_j]);
                        //if (i == 0 && m ==3 && j == 1) {
                        //    printf("%f",_cov.real());
                        //}
                    }
                }
            }
            cov[(i*nimages+m)*nimages+j] = _cov;
            //if ( i == 0 && m==3 && j ==1 ) printf("%d",((i*nimages+m)*nimages+j));
            _amp2_m = sqrt(_amp2_m*_amp2_j);
            coh[(i*nimages+m)*nimages+j] = _cov/_amp2_m;
        }
    }
    ''',
    name = 'co_mat_kernel',reduce_dims = False,no_return=True
)

In [ ]:
#| export
def co_mat(rslc:cp.ndarray, # rslc stack, dtype: `cupy.complexfloating`
            is_shp:cp.ndarray, # shp bool, dtype: `cupy.bool`
            block_size:int=128, # the CUDA block size, it only affects the calculation speed
            )-> tuple: # the covariance and coherence matrix
    nlines, width, nimages = rslc.shape
    az_win, r_win = is_shp.shape[-2:]
    az_half_win = (az_win-1)//2
    r_half_win = (r_win-1)//2

    cov = cp.zeros((nlines,width,nimages,nimages),dtype=rslc.dtype)
    coh = cp.empty((nlines,width,nimages,nimages),dtype=rslc.dtype)

    _co_mat_kernel(rslc, is_shp, cp.int32(nlines),cp.int32(width),cp.int32(nimages),
                    cp.int32(az_half_win),cp.int32(r_half_win),cov,coh,size = nlines*width,block_size=block_size)
    return cov,coh

The `rslc` is a three dimentional cupy `ndarray`. The `dtype` should be `cupy.complex64`. From outerest to innerest, the three dimentions are azimuth, range and image.
`is_shp` is a four dimentional cupy `ndarray`. From outerest ot innerest, they are azimuth, range, secondary pixel relative azimuth, secondary pixel relative range.

Here is an example:

In [ ]:
#import cupy as cp
rslc = cp.load('../../data/rslc.npy')
rslc = rslc[:5,:10,:5]
rslc.shape

(5, 10, 5)

`rslc` is a stack of 5 rslc images. Each of the image has 5 pixel in azimuth dimention and 10 pixels in range dimention.
Apply ks test on it:

In [ ]:
rmli = cp.abs(rslc)**2
sorted_rmli = cp.sort(rmli,axis=-1)
dist, p = ks_test(sorted_rmli,az_half_win=1,r_half_win=1)

Seclect SHP based on p value:

In [ ]:
is_shp = (p < 0.005) & (p >= 0.0)

Estimate the covarience and coherence matrix: 

In [ ]:
cov,coh = co_mat(rslc,is_shp)
cov.shape, coh.shape

((5, 10, 5, 5), (5, 10, 5, 5))

Both `cov` and `coh` are complex data. The shape shows each covarience or coherence matrix is 5 by 5 since there are 5 images.
And `cov` and `coh` are matrix for all 5*10 pixels.

In [ ]:
#| hide
# test

# az, r, image, image
half_az_win = is_shp.shape[2]//2;
half_r_win = is_shp.shape[3]//2;
for i, j, k, l in itertools.product(range(rslc.shape[0]),range(rslc.shape[1]),range(rslc.shape[2]),range(rslc.shape[2])):
    _cov = 0.0+0.0j
    _amp2_k = 0.0
    _amp2_l = 0.0
    # shp_az, shp_r
    for m, n in itertools.product(range(is_shp.shape[2]),range(is_shp.shape[3])):
        if is_shp[i,j,m,n]:
            _cov += rslc[i+m-half_az_win,j+n-half_r_win,k]*rslc[i+m-half_az_win,j+n-half_r_win,l].conj()
            _amp2_k += abs(rslc[i+m-half_az_win,j+n-half_r_win,k])**2
            _amp2_l += abs(rslc[i+m-half_az_win,j+n-half_r_win,l])**2
    assert abs(_cov-cov[i,j,k,l])<1.0e-7
    assert abs(_cov/math.sqrt(_amp2_k*_amp2_l) - coh[i,j,k,l]) < 1.0e-6

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()